<a href="https://colab.research.google.com/github/Yinlipp/SoccerTrackChallenge-2025_Colab/blob/main/stc_baseline_no_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ==========================================================================
# 📌 STC -2025 Inference & Evaluation Code
# The code is divided into two parts:
#
# Cell 1: Run Deep-EIoU on STC Video to Generate MOT-format Tracking Results
#     - Only the generated `.txt` result from this part needs to be submitted
#     - Estimated runtime: ~50 mins on Colab (T4 GPU)
#
# Cell 2: (Optional) Use HOTA to evaluate tracking results with GT
#     - Participants may IGNORE this part — it's for local evaluation only
# ==========================================================================

# ==========================================================================
# ✅ Cell 1: Run Deep-EIoU on STC Video to Generate MOT-format Tracking Results
# Only the generated .txt result from this cell needs to be submitted
# Estimated runtime: ~50 mins on Colab (T4 GPU)
# ============================================================================
# Install Python 3.7 and pip
!sudo apt-get install python3.7 python3.7-dev python3.7-distutils -y
!wget https://bootstrap.pypa.io/pip/3.7/get-pip.py
!python3.7 get-pip.py

# Install PyTorch (CUDA 11.6), and other dependencies
!python3.7 -m pip install torch==1.13.0+cu116 torchvision==0.14.0+cu116 \
torchaudio==0.13.0 --index-url https://download.pytorch.org/whl/cu116

!python3.7 -m pip install cython_bbox gdown \
https://github.com/KaiyangZhou/deep-person-reid/archive/master.zip

# Clone Deep-EIoU repo
!git clone https://github.com/hsiangwei0903/Deep-EIoU.git

# nstall requirements for ReID
%cd /content/Deep-EIoU/Deep-EIoU/reid
!python3.7 -m pip install -r requirements.txt

# Go back to main directory
%cd /content/Deep-EIoU/Deep-EIoU

# Download STC video and Deep-EIoU models (detector & ReID)
!gdown --fuzzy 'https://drive.google.com/file/d/1DQj5-kiU4VySymDXZuV84-nU1AZ4HZaj/view?usp=drive_link'  # video 117092.mp4
!gdown --fuzzy 'https://drive.google.com/file/d/1834kh10-X0Tu743fgmN7jXPVDKgq4ZqR/view?usp=drive_link' --output checkpoints/best_ckpt.pth.tar
!gdown --fuzzy 'https://drive.google.com/file/d/14zzlm1nI9Ws_Il9RYNChwPC7Fsul7xwl/view?usp=drive_link' --output checkpoints/sports_model.pth.tar-60

# ▶️ Run tracking inference
!python3.7 -m pip install https://github.com/KaiyangZhou/deep-person-reid/archive/master.zip  # torchreid
!python3.7 tools/demo.py --path 117092.mp4 --save_result True

# 📍Outputs:
# - Video: /content/Deep-EIoU/Deep-EIoU/YOLOX_outputs/yolox_x_ch_sportsmot/track_vis/(timestamp)/117092.mp4
# - Tracking TXT: /content/Deep-EIoU/Deep-EIoU/YOLOX_outputs/yolox_x_ch_sportsmot/track_vis/*.txt


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3.7 is already the newest version (3.7.17-1+jammy1).
python3.7-dev is already the newest version (3.7.17-1+jammy1).
python3.7-distutils is already the newest version (3.7.17-1+jammy1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.
--2025-04-17 03:25:06--  https://bootstrap.pypa.io/pip/3.7/get-pip.py
Resolving bootstrap.pypa.io (bootstrap.pypa.io)... 151.101.0.175, 151.101.64.175, 151.101.128.175, ...
Connecting to bootstrap.pypa.io (bootstrap.pypa.io)|151.101.0.175|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2636033 (2.5M) [text/x-python]
Saving to: ‘get-pip.py’

get-pip.py          100%[===================>]   2.51M  --.-KB/s    in 0.04s   

2025-04-17 03:25:06 (61.4 MB/s) - ‘get-pip.py’ saved [2636033/2636033]

  Using cached pip-24.0-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.0-py3-none-any.whl (2.1 MB)
  Attempting uninst

In [4]:

# ==========================================================
# Cell 2 (Optional): Compute HOTA metric with Ground Truth
# Official repo: https://github.com/JonathonLuiten/TrackEval
# (Participants may skip this cell; only for local evaluation)
# ==========================================================

# Clone TrackEval and install dependencies
%cd /content/
!git clone https://github.com/JonathonLuiten/TrackEval.git
!python3.7 -m pip install lap cython motmetrics
%cd /content/TrackEval/
!python3.7 -m pip install -r requirements.txt

# Prepare directories for GT and predictions
!mkdir -p /content/TrackEval/data/gt/mot_challenge/mydataset/117092/gt
!mkdir -p /content/TrackEval/data/trackers/mot_challenge/mydataset/my_tracker/data

# Download GT file (ground truth)
!gdown --fuzzy 'https://drive.google.com/file/d/1cQeC4c0FG8i8Xayw3LdzhEU5p42s7zD1/view?usp=drive_link' \
--output /content/TrackEval/data/gt/mot_challenge/mydataset/117092/gt/gt.txt

# Copy prediction results from Deep-EIoU
import glob, shutil, os
files_path = glob.glob('/content/Deep-EIoU/Deep-EIoU/YOLOX_outputs/yolox_x_ch_sportsmot/track_vis/*.txt')
for file in files_path:
    shutil.copy(file, "/content/TrackEval/data/trackers/mot_challenge/mydataset/my_tracker/data/117092.txt")

# Create seqmap (defines which sequences are used)
seqmap_path = "/content/TrackEval/data/gt/mot_challenge/mydataset/seqmaps/"
os.makedirs(seqmap_path, exist_ok=True)
with open(os.path.join(seqmap_path, "mydataset-test.txt"), "w") as f:
    f.write("name\n117092")

# Generate seqinfo.ini (basic video metadata)
ini_content = (
    "[Sequence]\n"
    "name=117092\n"
    "imDir=img1\n"
    "frameRate=30\n"
    "seqLength=1000000\n"
    "imWidth=1920\n"
    "imHeight=1080\n"
    "imExt=.jpg"
)
with open("/content/TrackEval/data/gt/mot_challenge/mydataset/117092/seqinfo.ini", "w") as f:
    f.write(ini_content)

# Format .txt files to comply with TrackEval (frame starts at 1, class ID = 1, space-separated)
def update_file_in_place(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    with open(file_path, 'w', encoding='utf-8') as f:
        for line in lines:
            parts = line.strip().split(',')
            if parts:
                if parts[0].isdigit():
                    parts[0] = str(int(parts[0]) + 1)
                if len(parts) >= 3:
                    parts[-3] = '1'  # Set class ID = 1
                f.write(' '.join(parts) + '\n')
            else:
                f.write(line)

# Apply formatting to GT and prediction result files
gt_txt = '/content/TrackEval/data/gt/mot_challenge/mydataset/117092/gt/gt.txt'
pred_txt = '/content/TrackEval/data/trackers/mot_challenge/mydataset/my_tracker/data/117092.txt'
update_file_in_place(gt_txt)
update_file_in_place(pred_txt)

# Run evaluation
%cd /content/TrackEval
!python3.7 scripts/run_mot_challenge.py \
  --BENCHMARK mydataset \
  --SPLIT_TO_EVAL test \
  --GT_FOLDER data/gt/mot_challenge/mydataset \
  --TRACKERS_FOLDER data/trackers/mot_challenge/mydataset \
  --TRACKERS_TO_EVAL my_tracker \
  --METRICS HOTA CLEAR Identity \
  --SKIP_SPLIT_FOL True


/content
fatal: destination path 'TrackEval' already exists and is not an empty directory.
/content/TrackEval
  Using cached numpy-1.18.1-cp37-cp37m-manylinux1_x86_64.whl.metadata (2.1 kB)
  Using cached scipy-1.4.1-cp37-cp37m-manylinux1_x86_64.whl.metadata (2.0 kB)
  Using cached pycocotools-2.0.2.tar.gz (23 kB)
  Preparing metadata (setup.py) ... done
  Using cached matplotlib-3.2.1-cp37-cp37m-manylinux1_x86_64.whl.metadata (5.2 kB)
  Using cached opencv_python-4.4.0.46-cp37-cp37m-manylinux2014_x86_64.whl.metadata (16 kB)
  Using cached scikit_image-0.16.2-cp37-cp37m-manylinux1_x86_64.whl.metadata (3.1 kB)
  Using cached pytest-6.0.1-py3-none-any.whl.metadata (7.4 kB)
  Using cached Pillow-8.1.2-cp37-cp37m-manylinux1_x86_64.whl.metadata (7.1 kB)
  Using cached tqdm-4.64.0-py2.py3-none-any.whl.metadata (57 kB)
  Using cached networkx-2.6.3-py3-none-any.whl.metadata (5.0 kB)
  Using cached PyWavelets-1.3.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinu